In [9]:
"""
TOPSIS法
"""
import numpy as np
import typing as ty
import pandas as pd


In [10]:
Grades = np.array([89,60,74,99])
Grades_Method1 = (Grades-min(Grades))/(max(Grades)-min(Grades))
Grades_Method1 = Grades_Method1/sum(Grades_Method1)
print(Grades_Method1)
Grades_Method2 = (Grades-0)/(100-0)
Grades_Method2 = Grades_Method2/sum(Grades_Method2)
print(Grades_Method2)


[0.35365854 0.         0.17073171 0.47560976]
[0.27639752 0.1863354  0.22981366 0.30745342]


In [11]:
'''
标准化处理
'''
Scores = np.array([[89.,1],[60,3],[74,2],[99,0]])#加上一个.表示浮点数类型，而不是整数类型
tmp = np.zeros(Scores.shape[1])
for i in range(Scores.shape[0]):
    for j in range(Scores.shape[1]):
        tmp[j] += Scores[i][j]**2
tmp = np.sqrt(tmp)
for i in range(Scores.shape[0]):
    for j in range(Scores.shape[1]):
        Scores[i][j]/=tmp[j]
Scores
# for i in range(Scores.shape[0]):
#     for j in range(Scores.shape[1]):
#         print(f'{Scores[i][j]:.4f}',end="&")
#     print("\\\\")

array([[0.54367431, 0.26726124],
       [0.36652201, 0.80178373],
       [0.45204381, 0.53452248],
       [0.60476132, 0.        ]])

In [12]:
'''
计算得分
'''
Z_min = np.copy(Scores[0])
Z_max = np.copy(Scores[0])
for i in range(Scores.shape[0]):
    for j in range(Scores.shape[1]):
        Z_min[j] = min(Z_min[j],Scores[i][j])
        Z_max[j] = max(Z_max[j],Scores[i][j])
tmp = np.ones(4)
for i in range(Scores.shape[0]):
    D_min = np.copy(Scores[i])-Z_min
    D_max = np.copy(Scores[i])-Z_max
    D_min = np.float_power(D_min,2)
    D_max = np.float_power(D_max,2)
    print(f"{np.sum(D_min)**0.5/(np.sum(D_max)**0.5+np.sum(D_min)**0.5):.4f}")
    tmp[i] = np.sum(D_min)**0.5/(np.sum(D_max)**0.5+np.sum(D_min)**0.5)
#归一化
tmp/=np.sum(tmp)
tmp

0.3734
0.7709
0.6375
0.2291


array([0.1857001 , 0.38337054, 0.31701594, 0.11391343])

In [13]:
"""
20条河流的水质情况数据
"""
df = pd.read_excel("20条河流的水质情况数据.xlsx")
df

,河流,含氧量（ppm),PH值,细菌总数(个/mL),植物性营养物量（ppm)
0,A,4.69,6.59,51,11.94
1,B,2.03,7.86,19,6.46
2,C,9.11,6.31,46,8.91
3,D,8.61,7.05,46,26.43
4,E,7.13,6.50,50,23.57
5,F,2.39,6.77,38,24.62
6,G,7.69,6.79,38,6.01
7,H,9.30,6.81,27,31.57
8,I,5.45,7.62,5,18.46
9,J,6.19,7.27,17,7.51


In [14]:
"""
原始矩阵正向化
"""
#PH值，中间型指标7
x_best = 7
M = max(np.abs(np.array(df.iloc[:,2])-x_best))
for i in range(df.shape[0]):
    df.iloc[i,2] = 1-abs(df.iloc[i,2]-x_best)/M
#细菌总数，极小型指标
M = max(np.array(df.iloc[:,3]))
for i in range(df.shape[0]):
    df.iloc[i,3] = M-df.iloc[i,3]
#植物性营养物量，区间型指标[10,20]
a = 10
b = 20
M = max(a-min(np.array(df.iloc[:,4])),max(np.array(df.iloc[:,4]))-b)
for i in range(df.shape[0]):
    if df.iloc[i,4]<a:
        df.iloc[i,4] = 1-(a-df.iloc[i,4])/M
    elif df.iloc[i,4]>b:
        df.iloc[i,4] = 1-(df.iloc[i,4]-b)/M
    else:
        df.iloc[i,4] = 1
df

,河流,含氧量（ppm),PH值,细菌总数(个/mL),植物性营养物量（ppm)
0,A,4.69,0.717241,3,1.000000
1,B,2.03,0.406897,35,0.694036
2,C,9.11,0.524138,8,0.905791
3,D,8.61,0.965517,8,0.444252
4,E,7.13,0.655172,4,0.691443
5,F,2.39,0.841379,16,0.600691
6,G,7.69,0.855172,16,0.655143
7,H,9.30,0.868966,27,0.000000
8,I,5.45,0.572414,49,1.000000
9,J,6.19,0.813793,37,0.784788


In [15]:
"""
正向化矩阵标准化
"""
for j in range(1,df.shape[1]):
    x = np.array(df.iloc[:,j])
    x = sum(np.float_power(x,2))**0.5
    for i in range(df.shape[0]):
        df.iloc[i,j] = df.iloc[i,j]/x
df


,河流,含氧量（ppm),PH值,细菌总数(个/mL),植物性营养物量（ppm)
0,A,0.162186,0.248255,0.024544,0.306458
1,B,0.070200,0.140837,0.286347,0.212693
2,C,0.315035,0.181417,0.065451,0.277586
3,D,0.297744,0.334190,0.065451,0.136145
4,E,0.246564,0.226772,0.032725,0.211898
5,F,0.082649,0.291223,0.130902,0.184086
6,G,0.265930,0.295997,0.130902,0.200773
7,H,0.321605,0.300771,0.220896,0.000000
8,I,0.188468,0.198127,0.400886,0.306458
9,J,0.214058,0.281674,0.302710,0.240504


In [16]:
"""
计算得分并归一化
"""
df['Score'] = 0
Z_max = np.array([max(df.iloc[:,1]),max(df.iloc[:,2]),max(df.iloc[:,3]),max(df.iloc[:,4])])
Z_min = np.array([min(df.iloc[:,1]),min(df.iloc[:,2]),min(df.iloc[:,3]),min(df.iloc[:,4])])
print(Z_max)
print(Z_min)
for i in range(df.shape[0]):
    tmp = np.array(df.iloc[i,1:-1]).astype("float64")#y?
    D_max = np.sum(np.float_power(tmp-Z_max,2))**0.5
    D_min = np.sum(np.float_power(tmp-Z_min,2))**0.5
    df.iloc[i,5] = D_min/(D_min+D_max)
#归一化
df.iloc[:,5] /= sum(df.iloc[:,5])
df

[0.32160534 0.3341898  0.4008859  0.30645756]
[0.06950825 0.         0.         0.        ]


,河流,含氧量（ppm),PH值,细菌总数(个/mL),植物性营养物量（ppm),Score
0,A,0.162186,0.248255,0.024544,0.306458,0.045058
1,B,0.070200,0.140837,0.286347,0.212693,0.047799
2,C,0.315035,0.181417,0.065451,0.277586,0.048497
3,D,0.297744,0.334190,0.065451,0.136145,0.048820
4,E,0.246564,0.226772,0.032725,0.211898,0.043113
5,F,0.082649,0.291223,0.130902,0.184086,0.044838
6,G,0.265930,0.295997,0.130902,0.200773,0.053947
7,H,0.321605,0.300771,0.220896,0.000000,0.050998
8,I,0.188468,0.198127,0.400886,0.306458,0.068074
9,J,0.214058,0.281674,0.302710,0.240504,0.068393
